In [ ]:
import numpy as np
import cv2
import os
from os.path import join
import tensorflow as tf

from modules.datasets import ImageTargetDataset, RandomConcatDataset
from modules.segm_transforms import train_transforms, test_transforms, ToTensorColor
from modules.metrics import FbSegm
from train.train import Model

import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
train_batch_size = 16
val_batch_size = 16
INPUT_SIZE = (224, 224)
AUG_PARAMS = [0.75, 1.25, 0.75, 1.25, 0.6, 1.4]
ANG_RANGE = 15

In [ ]:
train_trns = train_transforms(dataset='picsart', scale_size=INPUT_SIZE, ang_range=ANG_RANGE,
                                      augment_params=AUG_PARAMS, add_background=False,
                                      crop_scale=0.02)
val_trns = test_transforms(dataset='picsart', scale_size=INPUT_SIZE)

In [ ]:
data_dirs_hq = [
    '/workdir/data/datasets/picsart/',
    '/workdir/data/datasets/supervisely_person/',
]

data_dirs_coco = [
    '/workdir/data/datasets/coco_person/'
#     '/workdir/data/datasets/cityscapes_person/',
]


In [ ]:
train_dirs_hq = [join(d, 'train') for d in data_dirs_hq]
val_dirs_hq = [join(d, 'val') for d in data_dirs_hq]
train_dirs_coco = [join(d, 'train') for d in data_dirs_coco]
val_dirs_coco = [join(d, 'val') for d in data_dirs_coco]

In [ ]:
train_dataset_hq = ImageTargetDataset(train_dirs_hq,
                                           train_batch_size,
                                           shuffle=True,
                                           device='GPU:0',
                                           **train_trns,
                                           IMG_EXTN='.jpg',
                                           TRGT_EXTN='.png')
val_dataset_hq = ImageTargetDataset(val_dirs_hq,
                                           val_batch_size,
                                           shuffle=False,
                                           device='GPU:0',
                                           **val_trns,
                                           IMG_EXTN='.jpg',
                                           TRGT_EXTN='.png')

In [ ]:
train_dataset_coco = ImageTargetDataset(train_dirs_coco,
                                           train_batch_size,
                                           shuffle=True,
                                           device='GPU:0',
                                           **train_trns,
                                           IMG_EXTN='.jpg',
                                           TRGT_EXTN='.png')
val_dataset_coco = ImageTargetDataset(val_dirs_coco,
                                           val_batch_size,
                                           shuffle=False,
                                           device='GPU:0',
                                           **val_trns,
                                           IMG_EXTN='.jpg',
                                           TRGT_EXTN='.png')

In [ ]:
print(len(train_dataset_hq))
print(len(train_dataset_coco))

In [ ]:
train_dataset = RandomConcatDataset([train_dataset_hq, train_dataset_coco],
                                    [0.8,0.2], size=3580)
# train_dataset = train_dataset_hq

In [ ]:
for x in train_dataset:
    img, target = x[0], x[1]
    for i in range(8):
        print(img[i].shape)
        print(target[i].shape)
        plt.imshow(img[i])
        plt.show()
        plt.imshow(np.squeeze(target[i]))
        plt.show()
    break

In [ ]:
for x in val_dataset_hq:
    img, target = x[0], x[1]
    for i in range(8):
        print(img[i].shape)
        print(target[i].shape)
        plt.imshow(img[i])
        plt.show()
        plt.imshow(np.squeeze(target[i]))
        plt.show()
    break

In [ ]:
# Initialize model params
device = 'GPU:0'
model_name = 'mobilenet_small'
# model_name = 'test_model'
n_class=1
# old_model_path = '/workdir/data/experiments/test/model_best_0.1734900027513504.h5'#32
# old_model_path = '/workdir/data/experiments/fb_combined_mobilenetv3_tf/model_best_0.11067000031471252.h5'
# old_model_path = '/workdir/data/experiments/fb_combined_mobilenetv3_tf_coco_pixart_supervisely/model_best_0.16210000216960907.h5'
# old_model_path = '/workdir/data/experiments/fb_combined_mobilenetv3_tf_coco_pixart_supervisely/model_best_0.16161000728607178.h5'
# old_model_path = '/workdir/data/experiments/fb_combined_mobilenetv3_tf_coco_pixart_supervisely/model_best_0.15041999518871307.h5'
old_model_path = '/workdir/data/experiments/fb_combined_mobilenetv3_tf_coco_pixart_supervisely/model_best_0.14879000186920166.h5'

In [ ]:
# Train params
n_train = len(train_dataset)
n_val = len(val_dataset_hq)
# loss_name='bce'
loss_name = 'fb_combined'
optimizer = 'Adam'
lr = 0.00001
batch_size = train_batch_size
max_epoches = 1000
save_directory = '/workdir/data/experiments/fb_combined_mobilenetv3_tf_coco_pixart_supervisely'
reduce_factor = 0.75
epoches_limit = 3
early_stoping = 100
metrics = [FbSegm(channel_axis=-1)]

In [ ]:
mobilenet_model = Model(device=device,
                        model_name=model_name,
                        n_class=n_class,
                        input_shape=(1,224,224,3),
                        old_model_path=old_model_path)

In [ ]:
mobilenet_model.prepare_train(train_loader=train_dataset,
                              val_loader=val_dataset_hq,
                              n_train=n_train,
                              n_val=n_val,
                              loss_name=loss_name,
                              optimizer=optimizer,
                              lr = lr,
                              batch_size = batch_size,
                              max_epoches = max_epoches,
                              save_directory = save_directory,
#                               use_bce=True,
                              reduce_factor=reduce_factor,
                              epoches_limit=epoches_limit,
                              early_stoping=early_stoping,
                              metrics=metrics)

In [ ]:
mobilenet_model.fit()

In [ ]:
mobilenet_model.validate(val_loader, n_val)

# Test on some images

In [ ]:
test_imgs = os.listdir('/workdir/data/test_examples/')

In [ ]:
for img_path in test_imgs:
    img_path = os.path.join('/workdir/data/test_examples/', img_path)
    test_img = cv2.imread(img_path)
    test_img = test_img[:,:,::-1]
    test_img = cv2.resize(test_img, (224,224))
    test_tensor = ToTensorColor()(test_img)
    test_tensor = tf.expand_dims(test_tensor, 0)
    out = mobilenet_model.predict(test_tensor)
    print(out.shape)
    out_img = np.squeeze(out)#.numpy())
    print(out_img.shape)
    plt.imshow(test_img)
    plt.imshow((out_img>0.5)*255, alpha=0.4)
    plt.show()

# Convert model to TFLite

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(mobilenet_model.model)

In [ ]:
tflite_model = converter.convert()

In [ ]:
open("/workdir/data/converted_tflite_models/converted_model.tflite",
     "wb").write(tflite_model)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(mobilenet_model.model)

In [ ]:
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]

In [ ]:
tflite_model_weights_quantized = converter.convert()

In [ ]:
open("/workdir/data/converted_tflite_models/converted_model_weights_quantized.tflite",
     "wb").write(tflite_model_weights_quantized)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(mobilenet_model.model)

In [ ]:
def representative_dataset_generator():
    for input_value, target_value in val_dataset_hq:
            yield [input_value]
#         yield [tf.expand_dims(input_value[0],0)]



In [ ]:
# representative_dataset_gen = val_dataset_hq
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset_generator = representative_dataset_generator()

In [ ]:
tflite_model_full_quantized = converter.convert()

In [ ]:
open("/workdir/data/converted_tflite_models/converted_model_full_quantized.tflite",
     "wb").write(tflite_model_full_quantized)